# 1. Collecting data

## 1.1 Importing useful libraries

In [1]:
from concurrent.futures import ThreadPoolExecutor
import concurrent
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import re
import time
pd.set_option('display.max_columns', None)

In [2]:
provincias_dict = {
    "north": ["coruna+a", "lugo", "ourense", "pontevedra","asturias","cantabria","alava", "guipuzcoa", "vizcaya","navarra","rioja+la"],
    "north_centre":  ["avila", "burgos", "leon", "palencia", "salamanca", "segovia", "soria", "valladolid", "zamora","huesca", "teruel", "zaragoza"],
    "south_centre": ["badajoz", "caceres","albacete", "ciudad+real", "cuenca", "guadalajara", "toledo"],
    "south": ["almeria", "cadiz", "cordoba", "granada", "huelva", "jaen", "malaga","palmas+la", "tenerife"],
    "east": ["murcia", "castellon", "valencia","girona", "lleida", "tarragona", "illes+balears"],
    "big_capitals": ['madrid','barcelona','sevilla','alicante'],
    "prueba": ['huesca','teruel']
}

## 1.2 Custom functions to collect data

In [3]:
def connect_to(link, sleeptime, max_tries=3):
    """
    Attempts to connect to a car listing. Some links are mispelled (it's the web fault),
    so we could have an error. The function tries to connect a max number of #tries.
    Args:
    * link (string): The link of the car listing.
    * sleeptime (float): The pproximated number seconds to sleep between connections.
    * max_tries (int): The max. number of times it tries to connect.
    Returns:
    * response or string: If there was a connection, the return is a response. Otherwise it returns 'Failed'.
    """

    tries = 0
    while True:
        
        try: # connecting
            response = requests.get(link)
            break
                    
        except: # error connection
            tries += 1
            if tries > max_tries:
                return 'Failed'            
            print(' '*100,end='\r')
            print(f"({tries}) Reconnecting to web...", end='\r')
            sleep(abs(np.random.normal(sleeptime+tries,0.15)))
            continue
    
    return response

In [4]:
def extract_feature(css_path,numerical,soup):
    """
    Extracts a feature from the html of a car listing.
    Args:
    * css_path (string): The css path towards the feature.
    * numerical (boolean): Equal to True if the feature is numerical, otherwise false.
    * soup (bs4.BeautifulSoup): Contains the html information of the car listing.
    Returns:
    * float or string: If the feature is numerical, it returns a float. Otherwise it returns a string.
    """
    if numerical == 'True':
        feature = soup.select(css_path)[0]
        feature = re.findall(r'\d+[.]?[,]?\d*',feature.get_text().strip())[0].replace('.','').replace(',','.')
    elif numerical == 'False':
        feature = soup.select(css_path)[0].get_text().strip()
    
    return feature

In [5]:
def get_sales_data(location,car_class,n_pages,sleeptime):
    """
    Gets the sales data for a car class in a specific location.
    Args:
    * location (string): The location (province).
    * car_class (string): The car class ('standard','sport','commercial' or '4x4').
    * n_pages (int or string): The number of pages that will be read. Use 'auto' to automatically read all.
    Returns:
    * pd.DataFrame: Table containing the sales data for the specified conditions.
    """
    
    # We create the empty database where we'll add car samples.
    data = pd.DataFrame(columns=['year','cv','km','fuel','doors','gearbox','emissions',\
                                 'color','warranty','seller','id','brand','price','boot',\
                                 'length','height','width','seats','max_sp','cmixto',\
                                'curban','extraurban','0-100','autonomy','displac','cylinders',\
                                'transmission','max_par','gear','class','location','link'])

    
    # Reading CSS Path for features from the HTML table.
    with open("../data/css_paths.txt", "r") as f:
        css_paths = json.load(f)
        
    paths = [list_css[0] for list_css in css_paths.values()] 
    # The following list states if the feature is numerical ('True'), otherwise 'False'
    paths_bool = [list_css[1] for list_css in css_paths.values()] 
    
    # Rewriting car_classes for web search compatibility.
    if car_class == 'standard':
        class_string = 'pequeno-mediano-grande-familiar-monovolumen'
    elif car_class == '4x4':
        class_string = 'coches-4x4-todoterreno'
    elif car_class == 'commercial':
        class_string = 'furgonetas-segunda-mano'
    elif car_class == 'sport':
        class_string = 'coches-deportivos-segunda-mano'
    else:
        print('ERROR. The provided class does not exist. Please choose "standard","4x4","commercial" or "sport"')
        return
    
    
    ncar = 0 # Numbers of cars added to the database.
    skipped = 0 # Number of skipped cars.
    
    # This is the baselink, from which we we'll create each page link:
    baselink = "https://www.coches.com/coches-segunda-mano/"+class_string+"-en-"+location+".htm?page="
    
    # If n_pages == 'auto', we will search 499 pages (the maximum available). Otherwise, we go for the specified number.
    max_page = 499
    if n_pages != 'auto':
        max_page = n_pages
        
    # --- ITERATION OVER EACH PAGE ---
    for i in range(max_page):
        
        # We create the link for this page
        link = baselink+str(i)      
        
        # Connecting to the web page
        while True:
            try:
                response = requests.get(link)
                break
                
            except:
                print(' '*100,end='\r')
                print('Reconnecting to next page...', end='\r')
                sleep(abs(np.random.normal(sleeptime+4,0.15)))
                continue
        
        # Extracting HTML and searching for cars posted in the page.
        soup_page = BeautifulSoup(response.content, "html.parser")
        sleep(abs(np.random.normal(sleeptime,0.15)))
        car_links = soup_page.select("html body main.content-page div#vo-results.vo-results--rebranding div.pillList.vo-results__card-list.script__vo-results-card-list div.cc-car-card.vo-results__card.pill.script__pill")
        

        # --- ITERATION OVER EACH CAR --- 
        for car_link in car_links:
            
            if car_link.a['href'] == '':
                break # If this card has no car associated, move on
            
            # Extract ID from the car
            car_id = re.findall('id=([\d]+)', car_link.a['href'])[0]
            
            # Connect to the car listing (try 2 times, otherwise move on)   
            response = connect_to(car_link.a['href'],sleeptime=sleeptime)
            if response == 'Failed':
                skipped += 1
                print(' '*100,end='\r')
                print(f'Skipping one car (total = {skipped})',end='\r')
                break
            
            # Extract HTML from the car listing
            soup = BeautifulSoup(response.content, "html.parser")
            sleep(abs(np.random.normal(sleeptime,0.15)))
            
            # Read basic features
            features_html = soup.select(".cc-car-overview")
            if len(features_html)==0:
                break # If we can't find the basic features, skip the car
                
            features = features_html[0].select("p")
            car_features = [code.get_text() for i,code in enumerate(features) if (i+1)%2==0]

            # Read the car brand from the title
            car_brand = soup.select("h1.index-card__make-model")[0].get_text()
            car_brand = re.findall(r"[\w]+",car_brand)[0]
                
            # Obtain the price as the max of the prices (we are not interested in prices associated with bank loans)
            car_price = [a.get_text() for a in soup.select("div.index-card__price-number")] 
            car_prices = [int(re.findall(r'\d+[.]*\d*',price)[0].replace(".","")) for price in car_price if len(re.findall(r'\d+[.]*\d*',price))>0]
            
            # Read additional features              
            some_features = [extract_feature(path,var_bool,soup=soup) for path, var_bool in zip(paths,paths_bool)]
            
            
            # Concatenating them to form the new list with the car features
            more_features = [car_id, car_brand, np.max(car_prices)]+some_features+[car_class, location, car_link.a['href']]
            car_features = car_features + more_features
            
            # Storing the car features in the database
            data.loc[ncar] = car_features
            ncar += 1
            
        # --- FINISHED ITERATION OVER EACH CAR --- 
        
        if n_pages == 'auto':
            
            # Check what's the next page link
            try:
                next_page = soup_page.select(".pager-next")[0].a['href']
            except:
                # sometimes we only have 1 page, and so there's no next button.In that case, we're also in the last page:
                next_page = link
            
            # Check if we reached the limit
            if next_page == link:
                break
        
        else:
            # Show progress if n_pages != 'auto'
            print(' '*100,end='\r')
            print(f'{(round((i+1)/n_pages*100,1))}% complete ({data.shape[0]} items).',end='\r')
            
    # --- FINISHED ITERATION OVER EACH PAGE ---
    
    print('                                                                                ',end='\r')
    print(f"Obtained {data.shape[0]} items of class '{car_class}' in {location.capitalize()}.",end='\r')   
    return data

In [6]:
def get_regional_data(location,n_pages,sleeptime):
    car_classes = ['sport','commercial','4x4','standard']
    
    car_data = [get_sales_data(location=location,car_class=car_class, n_pages = n_pages, sleeptime = sleeptime) for car_class in car_classes]
    
    car_data = pd.concat(car_data,axis=0).reset_index(drop=True)
    
    print(f"Regional data from {location.capitalize()} scanned succesfully for a total of {car_data.shape[0]} items.")   
    
    return car_data

In [7]:
def get_community_data(comunidad, n_pages = 'auto', sleeptime = 1.5, save_csv = False):
    provincias = provincias_dict[comunidad]
    with ThreadPoolExecutor() as executor:
        reg_cars_futures = [executor.submit(lambda x: get_regional_data(x,sleeptime = 0, n_pages = n_pages), prov) for prov in provincias]
        reg_cars = [future.result() for future in concurrent.futures.as_completed(reg_cars_futures)]
        regional_cars = pd.concat(reg_cars,axis=0).reset_index(drop=True)
    
    if save_csv == True:
        regional_cars.to_csv('../data/'+comunidad+'.csv',index=False)
    print(f"All data from {comunidad.capitalize()} saved succesfully with a total of {regional_cars.shape[0]} items.")   
     
    return regional_cars

# South centre

In [8]:
%%time
south_centre = get_community_data('south_centre', sleeptime = 0, save_csv = True)

Regional data from Cuenca scanned succesfully for a total of 36 items.          
Regional data from Albacete scanned succesfully for a total of 4243 items.                          
Regional data from Caceres scanned succesfully for a total of 6214 items.                           
Regional data from Guadalajara scanned succesfully for a total of 6505 items.                       
Regional data from Badajoz scanned succesfully for a total of 7468 items.                           
Regional data from Toledo scanned succesfully for a total of 9450 items.                            
Regional data from Ciudad+real scanned succesfully for a total of 12688 items.                      
All data from South_centre saved succesfully with a total of 46604 items.
CPU times: user 5h 22min 52s, sys: 2min 53s, total: 5h 25min 46s
Wall time: 7h 21min 6s


In [9]:
south_centre

,year,cv,km,fuel,doors,gearbox,emissions,color,warranty,seller,id,brand,price,boot,length,height,width,seats,max_sp,cmixto,curban,extraurban,0-100,autonomy,displac,cylinders,transmission,max_par,gear,class,location,link
0,2018,258 CV,60.000 km,Diesel,5 puertas,automática secuencial,183 gr/m3,negro,No,Particular,7425029,Bmw,40330,550,491,170,199,5,230,7.0,10.0,5.2,6.7,0,2993,6 en línea,Automática secuencial,560,8,sport,cuenca,https://www.coches.com/coches-segunda-mano/oca...
1,2013,140 CV,170.000 km,Diesel,4 puertas,manual,119 gr/m3,azul,No,Particular,6981065,Volkswagen,13500,532,480,142,186,4,214,4.5,5.3,4.1,9.8,0,1968,4 en línea,Manual,320,6,sport,cuenca,https://www.coches.com/coches-segunda-mano/oca...
2,2018,102 CV,136.000 km,Diesel,4 puertas,manual,138 gr/m3,blanco,No,Particular,7473522,Volkswagen,22500,0,441,186,179,5,172,5.3,6.4,4.6,0.0,0,1968,4 en línea,Manual,250,5,commercial,cuenca,https://www.coches.com/coches-segunda-mano/oca...
3,2015,75 CV,300.000 km,Diesel,3 puertas,manual,123 gr/m3,blanco,No,Particular,7595937,Renault,4900,0,390,181,183,2,150,4.7,5.4,4.4,16.3,0,1461,4 en línea,Manual,180,5,commercial,cuenca,https://www.coches.com/coches-segunda-mano/oca...
4,1995,85 CV,100.000 km,Diesel,4 puertas,manual,0 gr/m3,blanco,No,Particular,7515358,Fiat,7000,0,0,0,200,6,0,0.0,0.0,0.0,0.0,0,2499,4 en línea,Manual,164,5,commercial,cuenca,https://www.coches.com/coches-segunda-mano/oca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46599,07/2011,115 CV,128.495 km,Diesel,5 puertas,manual,115 gr/m3,Blanco,SÍ,Profesional,7575155,Mazda,9200,340,446,147,176,5,186,4.3,5.2,3.8,12.0,0,1560,4 en línea,Manual,270,6,standard,ciudad+real,https://www.coches.com/coches-segunda-mano/oca...
46600,2018,75 CV,109.850 km,Diesel,5 puertas,manual,112 gr/m3,blanco,SÍ,Profesional,7568843,Citroen,9490,775,440,184,185,5,152,4.1,4.6,3.8,0.0,0,1499,4 en línea,Manual,230,5,standard,ciudad+real,https://www.coches.com/coches-segunda-mano/oca...
46601,02/2005,100 CV,137.362 km,Gasolina,3 puertas,manual,0 gr/m3,Negro,SÍ,Profesional,7562425,Renault,700,0,381,142,167,5,185,6.7,9.2,5.3,10.5,0,1390,4 en línea,Manual,127,5,standard,ciudad+real,https://www.coches.com/coches-segunda-mano/oca...
46602,05/2001,110 CV,170.812 km,Diesel,5 puertas,manual,0 gr/m3,Beige,SÍ,Profesional,7562293,Renault,900,0,458,143,177,5,0,5.4,7.2,4.4,12.3,0,1870,4 en línea,Manual,250,6,standard,ciudad+real,https://www.coches.com/coches-segunda-mano/oca...


# Big_capitals

In [10]:
big_capitals = get_community_data('big_capitals', sleeptime = 0, save_csv = True)

Regional data from Alicante scanned succesfully for a total of 9012 items.      
Regional data from Madrid scanned succesfully for a total of 21917 items.                           
Regional data from Barcelona scanned succesfully for a total of 21852 items.    
Regional data from Sevilla scanned succesfully for a total of 18640 items.      
All data from Big_capitals saved succesfully with a total of 71421 items.
